In [31]:
import pandas as pd
GOLD_PATH = './Medallion_Architecture/gold/'

In [32]:
df = pd.read_parquet('./Medallion_Architecture/silver/Silver.parquet')

In [33]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,loadDateTimeStamp
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,Polterpup,A,A,A,12,Drifter,-34,0,True,True,1-Up Mushroom,64,26,4,Mario Tennis Aces,2025-10-27 10:50:47.281938
1,Peach,Green Caps,Donut Plains,Circuit Special,Koopa Troopa,C,A,B,16,Drifter,149,4,True,True,"Red Shell, Super Star",335,40,5,Mario Tennis Aces,2025-10-27 10:50:47.281938
2,Waluigi,Unknown Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174,1,True,True,Green Shell,182,57,5,Mario Kart 8 Deluxe,2025-10-27 10:50:47.281938
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1,5,True,True,1-Up Mushroom,333,84,6,Super Mario Bros.,2025-10-27 10:50:47.281938
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,Toad,C,C,B,10,Blocker,28,2,True,True,"Red Shell, Banana Peel, Fire Flower",461,55,7,Super Mario World,2025-10-27 10:50:47.281938


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Player Name                   48000 non-null  string        
 1   Team                          48000 non-null  object        
 2   World                         48000 non-null  object        
 3   Vehicle Type                  48000 non-null  string        
 4   Companion                     48000 non-null  object        
 5   Kart Racing Rank              48000 non-null  string        
 6   Platforming Rank              48000 non-null  string        
 7   Boss Battle Rank              48000 non-null  string        
 8   Power-Ups Used                48000 non-null  int32         
 9   Kart Role                     48000 non-null  string        
 10  Team Points                   48000 non-null  int32         
 11  Lives Lost                  

In [35]:
# Recast string columns which were downgraded to object type by Pandas
recast_string_columns = ['Team', 'World', 'Companion']

df[recast_string_columns] = df[recast_string_columns].astype('string')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Player Name                   48000 non-null  string        
 1   Team                          48000 non-null  string        
 2   World                         48000 non-null  string        
 3   Vehicle Type                  48000 non-null  string        
 4   Companion                     48000 non-null  string        
 5   Kart Racing Rank              48000 non-null  string        
 6   Platforming Rank              48000 non-null  string        
 7   Boss Battle Rank              48000 non-null  string        
 8   Power-Ups Used                48000 non-null  int32         
 9   Kart Role                     48000 non-null  string        
 10  Team Points                   48000 non-null  int32         
 11  Lives Lost                  

In [37]:
rank_mapping = {'S': 5, 'A': 4, 'B': 3, 'C': 2, 'D': 1}
df['RankNumeric'] = df['Kart Racing Rank'].map(rank_mapping)

team_info = df.groupby('Team').agg(
    TotalTeamPoints=('Team Points', 'sum'),
    AverageRank=('RankNumeric', 'mean'),
    TotalCoinsSpent=('Coins Spent in Toad Town', 'sum')
).reset_index()

print("Total team points, average rank, and total coins spent in Toad Town by team:\n", team_info)
team_info.to_parquet(f'{GOLD_PATH}gold_team_info.parquet', index=False)

Total team points, average rank, and total coins spent in Toad Town by team:
                Team  TotalTeamPoints  AverageRank  TotalCoinsSpent
0      Dino Buddies           274062     3.000224          1363819
1        Green Caps           283310     3.012155          1411585
2        Koopa Clan           282558     2.987751          1375545
3  Mushroom Royalty           284478     2.999568          1438670
4          Red Caps           277777     2.975033          1409952
5  Sarasaland Stars           289633     3.030422          1426623
6      Toad Brigade           290411     3.040237          1395065
7        Tricksters           279585     3.037143          1394614
8      Unknown Team           138202     3.041718           741924


In [38]:
player_team_points = df.groupby('Player Name').agg(
    TotalTeamPoints=('Team Points', 'sum')
).reset_index().sort_values('TotalTeamPoints', ascending=False)

print("Total team points by player, in descending order:\n", player_team_points)
player_team_points.to_parquet(f'{GOLD_PATH}gold_player_team_points.parquet', index=False)

Total team points by player, in descending order:
        Player Name  TotalTeamPoints
9   Unknown Player           454698
3            Luigi           172753
1       Bowser Jr.           170904
0           Bowser           165782
10         Waluigi           165295
4            Mario           163519
2            Daisy           163110
6         Rosalina           162711
7             Toad           160752
5            Peach           159581
11           Wario           157165
12           Yoshi           153025
8         Toadette           150721


In [39]:
player_powerups = df.groupby('Player Name').agg(
    CountGames=('Power-Ups Used', 'count'),
    AvgPowerUpsUsed=('Power-Ups Used', 'mean')
).reset_index()

print("Total count and average # of power-ups used per player:\n", player_powerups)
player_powerups.to_parquet(f'{GOLD_PATH}gold_player_powerups.parquet', index=False)

Total count and average # of power-ups used per player:
        Player Name  CountGames  AvgPowerUpsUsed
0           Bowser        3268        17.407895
1       Bowser Jr.        3241        17.525455
2            Daisy        3294        17.472981
3            Luigi        3269        17.398287
4            Mario        3181        17.196793
5            Peach        3230        17.309598
6         Rosalina        3234        17.754484
7             Toad        3212        17.642590
8         Toadette        3295        17.386343
9   Unknown Player        9040        17.603540
10         Waluigi        3321        17.364950
11           Wario        3111        17.355513
12           Yoshi        3304        17.486077


In [40]:
world_lives_lost = df.groupby('World').agg(
    AvgLivesLost=('Lives Lost', 'mean')
).reset_index().sort_values('AvgLivesLost', ascending=False)

print("Average # of lives lost in each world, descending order:\n", world_lives_lost)
world_lives_lost.to_parquet(f'{GOLD_PATH}gold_lives_lost.parquet', index=False)

Average # of lives lost in each world, descending order:
               World  AvgLivesLost
0      Donut Plains      2.529310
2  Mushroom Kingdom      2.526588
4         Toad Town      2.512022
5    Yoshi's Island      2.503966
3        Star World      2.497933
1     Koopa Kingdom      2.477140


In [41]:
top_player_per_team = df.groupby(['Team', 'Player Name'])['Team Points'].sum().reset_index()
top_player_per_team = top_player_per_team.sort_values(['Team', 'Team Points'], ascending=[True, False])
top_players = top_player_per_team.groupby('Team').first().reset_index()

print("Top player per team by team points:\n", top_player_per_team)
top_player_per_team.to_parquet(f'{GOLD_PATH}gold_top_player_per_team.parquet', index=False)

Top player per team by team points:
              Team     Player Name  Team Points
9    Dino Buddies  Unknown Player        55720
0    Dino Buddies          Bowser        21605
3    Dino Buddies           Luigi        20234
1    Dino Buddies      Bowser Jr.        19463
10   Dino Buddies         Waluigi        18704
..            ...             ...          ...
108  Unknown Team           Mario         8985
107  Unknown Team           Luigi         8578
109  Unknown Team           Peach         8471
105  Unknown Team      Bowser Jr.         8305
115  Unknown Team           Wario         7883

[117 rows x 3 columns]


In [42]:
vehicle_popularity = df['Vehicle Type'].value_counts().reset_index()
vehicle_popularity.columns = ['Vehicle Type', 'Count']

print("Popularity of each vehicle type:\n", vehicle_popularity)
vehicle_popularity.to_parquet(f'{GOLD_PATH}gold_vehicle_popularity.parquet', index=False)

Popularity of each vehicle type:
       Vehicle Type  Count
0          Scooter   6916
1    Standard Kart   6914
2       Pipe Frame   6868
3       Biddybuggy   6863
4           Mach 8   6839
5       Comet Bike   6828
6  Circuit Special   6772


In [43]:
highest_risk_players = df.groupby('Player Name').agg(
    TotalLivesLost=('Lives Lost', 'sum')
).reset_index().sort_values('TotalLivesLost', ascending=False)

print("Players at highest risk - total lives lost:\n", highest_risk_players)
highest_risk_players.to_parquet(f'{GOLD_PATH}gold_highest_risk_players.parquet', index=False)

Players at highest risk - total lives lost:
        Player Name  TotalLivesLost
9   Unknown Player           22667
12           Yoshi            8367
8         Toadette            8353
10         Waluigi            8349
2            Daisy            8214
5            Peach            8166
1       Bowser Jr.            8160
3            Luigi            8103
6         Rosalina            8092
0           Bowser            8072
7             Toad            8070
4            Mario            8009
11           Wario            7754


In [44]:
levels_per_world = df.groupby('World').agg(
    TotalLevelsCompleted=('Levels Completed', 'sum'),
    AvgLevelsCompleted=('Levels Completed', 'mean')
).reset_index()

print("Average and total # of levels completed in each world:\n", levels_per_world)
levels_per_world.to_parquet(f'{GOLD_PATH}gold_levels_per_world.parquet', index=False)

Average and total # of levels completed in each world:
               World  TotalLevelsCompleted  AvgLevelsCompleted
0      Donut Plains                507269           62.734232
1     Koopa Kingdom                503194           62.687679
2  Mushroom Kingdom                491277           61.904864
3        Star World                496809           62.248966
4         Toad Town                501699           62.501433
5    Yoshi's Island                497014           62.572580


In [45]:
hit_by_enemies = df.groupby('Team').agg(
    TotalTimesHit=('Times Hit by Enemies', 'sum'),
    AvgTimesHit=('Times Hit by Enemies', 'mean')
).reset_index()

print("Total and average times each team was hit by enemies:\n", hit_by_enemies)
hit_by_enemies.to_parquet(f'{GOLD_PATH}gold_hit_by_enemies.parquet', index=False)

Total and average times each team was hit by enemies:
                Team  TotalTimesHit  AvgTimesHit
0      Dino Buddies          27417     4.977669
1        Green Caps          27943     4.967644
2        Koopa Clan          28023     5.037390
3  Mushroom Royalty          28368     4.966387
4          Red Caps          27914     4.954562
5  Sarasaland Stars          28708     5.031195
6      Toad Brigade          28246     4.959789
7        Tricksters          27619     4.934608
8      Unknown Team          14965     5.055743


In [46]:
total_coins_spent = df.groupby('Team').agg(
    TotalCoinsSpent=('Coins Spent in Toad Town', 'sum')
).reset_index()

print("Total coins spent in Toad Town by each team:\n", total_coins_spent)
total_coins_spent.to_parquet(f'{GOLD_PATH}gold_total_coins_spent.parquet', index=False)

Total coins spent in Toad Town by each team:
                Team  TotalCoinsSpent
0      Dino Buddies          1363819
1        Green Caps          1411585
2        Koopa Clan          1375545
3  Mushroom Royalty          1438670
4          Red Caps          1409952
5  Sarasaland Stars          1426623
6      Toad Brigade          1395065
7        Tricksters          1394614
8      Unknown Team           741924
